In [1]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

/Volumes/study/github/data-juicer/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_config = {
    "stop_tokens": ["<|im_start|>", "<|im_end|>", "<|endoftext|>"],
    "stop_token_ids": [151643, 151644, 151645],
    "stop_tokens_assistant": ["Assistant", "assistant"],
    "pre_query_template": "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n",
}
model_path = "../../../models/Qwen2.5-3B-Instruct"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
pipeline = transformers.pipeline(
    "text-generation", model=model_path, tokenizer=tokenizer, device="mps"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.72s/it]
Device set to use mps


In [4]:
pre_query_template = model_config["pre_query_template"]
print(pre_query_template)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user



In [5]:
instruction = pipeline(
    pre_query_template,
    max_new_tokens=2048,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature=1,
    top_p=1,
)

sanitized_instruction = instruction[0]["generated_text"][
    len(pre_query_template) :
].split("\n")[0]
print(f"Extracted Instruction: {sanitized_instruction}")

Extracted Instruction: If an alien visits Earth and sees the complex system of roads and public transportation used to transport people around cities, how might they interpret this as part of civilization?


In [6]:
model = AutoModelForCausalLM.from_pretrained(model_path).to("mps")
messages = [
    {
        "role": "system",
        "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant.",
    },
    {"role": "user", "content": sanitized_instruction},
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(**model_inputs, max_new_tokens=512)
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.73s/it]


In [7]:
print(f"User's message: {sanitized_instruction}")
print("-" * 100)
print(f"Assitant's response: {response}")

User's message: If an alien visits Earth and sees the complex system of roads and public transportation used to transport people around cities, how might they interpret this as part of civilization?
----------------------------------------------------------------------------------------------------
Assitant's response: An alien visitor might interpret the complex system of roads and public transportation in Earth's cities from several perspectives, considering their advanced technology and unique environment. Here’s how they might analyze it:

1. **Communication and Coordination**: The alien might observe that these systems are highly coordinated and well-maintained, indicating a sophisticated level of communication and organization. The ability to move large numbers of people efficiently suggests a strong social structure capable of planning and executing large-scale projects.

2. **Resource Management**: Public transportation systems often involve significant infrastructure and requi